# Programación Avanzada de Computadoras
## Laboratorio 3
## Sección: LS6B

---
## Parte 1: tkiner, Webscrapping - 13

Complete la siguiente aplicacion en tkinter para que contenga los datos de la cartelera local, utilizando la informacion disponible en la pagina: https://www.elseptimoarte.net/peliculas/

El boton Actualizar llenara el Combobox con los titlos de las peliculas de la cartelera

Al seleccionar un titulo, los detalles se mostrarán en los Entrys a la derecha y en el Text

---

In [1]:
import requests
from bs4 import BeautifulSoup
import sqlite3

# Se lee la informacion necesaria de la peliculas (pagina 1)
# Genere la siguiente lista: [(titulo, director, duracion, sinopsis, url), (titulo, director, duracion, sinopsis, url), ...]
url="https://www.elseptimoarte.net/peliculas"

r = requests.get (url)

soup= BeautifulSoup(r.text,'html.parser')      # Se obtienen mejores resultados con 'html.parser'
contenido= soup.find_all('h3')
links=[]
titulo=[]

for i in range (len(contenido)):
    
    try:
        links.append(url+contenido[i].a['href'])
        titulo.append(contenido[i].a['title'])
    except:
        pass

director=[]
duracion=[]
sinopsis=[]

for i in range (len(links)):

    url1=links[i]
    r1=requests.get(url1)
    soup1=BeautifulSoup(r1.text,'html.parser')
    director.append(soup1.find('p',class_='director').text)
    duracion.append(soup1.find('p',class_='duracion').text)
    sinopsis.append(soup1.find('div',class_='info').text)
    
    
lista_tuplas=[]
tupla=[]
    
for i in range(len(links)):

    tupla.append(titulo[i])
    tupla.append(director[i])
    tupla.append(duracion[i])
    tupla.append(sinopsis[i])
    tupla.append(links[i])
    lista_tuplas.append(tuple(tupla))
    tupla=[]
    

try :
    
    conn= sqlite3.connect("movies.db")
    cur=conn.cursor()
    cur.execute("DROP TABLE tabla ")
    
    cur.execute("CREATE TABLE IF NOT EXISTS tabla (titulo TEXT ,\
                                                   director TEXT,\
                                                   duracion TEXT,\
                                                   sinopsis TEXT,\
                                                   links TEXT)")
    conn.commit()
    
    cur.executemany("INSERT INTO tabla (titulo,director,duracion,sinopsis,links) VALUES (?,?,?,?,?)", lista_tuplas )
    
    conn.commit()
    conn.close()
    
except:
    
    cur.execute("CREATE TABLE IF NOT EXISTS tabla (titulo TEXT ,\
                                                   director TEXT,\
                                                   duracion TEXT,\
                                                   sinopsis TEXT,\
                                                   links TEXT)")
    conn.commit()
    
    cur.executemany("INSERT INTO tabla (titulo,director,duracion,sinopsis,links)VALUES (?,?,?,?,?)", lista_tuplas )
    
    conn.commit()
    conn.close()
    
    
print(links)

# Se carga la informacion en la tabla "peliculas" del archivo "movies.db".
# La tabla tiene los siguientes campos:
# - titulo
# - director
# - duracion
# - sinopsis
# - url

['https://www.elseptimoarte.net/peliculas/peliculas/soy-leyenda-2-1353.html', 'https://www.elseptimoarte.net/peliculas/peliculas/dos-policias-rebeldes-iii-4608.html', 'https://www.elseptimoarte.net/peliculas/peliculas/top-gun-2-5094.html', 'https://www.elseptimoarte.net/peliculas/peliculas/daredevil-2-2182.html', 'https://www.elseptimoarte.net/peliculas/peliculas/el-hombre-invisible-5989.html', 'https://www.elseptimoarte.net/peliculas/peliculas/g-i--joe-3-8031.html', 'https://www.elseptimoarte.net/peliculas/peliculas/jumper-2-1956.html', 'https://www.elseptimoarte.net/peliculas/peliculas/machete-kills-again\x85-in-space-8760.html', 'https://www.elseptimoarte.net/peliculas/peliculas/bloodshot-11143.html', 'https://www.elseptimoarte.net/peliculas/peliculas/beetlejuice-2-6462.html', 'https://www.elseptimoarte.net/peliculas/peliculas/sonic-the-hedgehog-14169.html', 'https://www.elseptimoarte.net/peliculas/peliculas/school-of-rock-2:-america-rocks-2015.html', 'https://www.elseptimoarte.net/

In [2]:
import os
import tkinter as tk
import tkinter.ttk as ttk
import textwrap
import sqlite3
import webbrowser




class App:
    def __init__(self, master):
        master.title("Cartelera - Cine")
            
        frm1 = tk.Frame(master)
        frm2 = tk.Frame(master)
        frm1.pack(side='left', padx=10, pady=10)
        frm2.pack(side='left', padx=10, pady=10)
        
        self.seleccione= tk.StringVar()
        self.movie_list = self.query_movie_list()
        self.titulo = tk.StringVar(value="Titulo")
        self.director = tk.StringVar(value="Director")
        self.duracion = tk.StringVar(value="Fecha")
                
        self.seleccione.set("Seleccione una pelicula...")    
            
        # ------------------ frm1 --------------------
        self.cboPeliculas = ttk.Combobox(frm1, width=50, values=self.movie_list,textvariable=self.seleccione)
        self.btnAbrirWeb = tk.Button(frm1, text="Ir a Web...", width=18, command=self.abrir_webbrowser, state='disable')
        self.btnSalir = tk.Button(frm1, text="Salir", width=18, command=master.destroy)
        self.lblBlank = tk.Label(frm1, text="", font="Arial 130 bold")
        
        self.cboPeliculas.grid(row=0, column=0, columnspan=2, padx=5, pady=5)
        self.btnAbrirWeb.grid(row=2, column=0, padx=5, pady=5)
        self.btnSalir.grid(row=2, column=1, padx=5, pady=5)
        self.lblBlank.grid(row=1, column=0, columnspan=2, padx=5, pady=5)
        
        # ------------------ frm2 ---------------------
        self.lblTitulo = tk.Label(frm2, text="Titulo")
        self.lblDirector = tk.Label(frm2, text="Director")
        self.lblDuracion = tk.Label(frm2, text="Duracion")
        self.lblSinopsis = tk.Label(frm2, text= "Sinopsis")
        
        self.entTitulo = tk.Entry(frm2, width=49, state='disable', textvariable=self.titulo)
        self.entDirector = tk.Entry(frm2, width=49, state='disable', textvariable=self.director)
        self.entDuracion = tk.Entry(frm2, width=49, state='disable', textvariable=self.duracion)
        self.scroll_Y = ttk.Scrollbar(frm2, orient='vertical')
        self.txtSinopsis = tk.Text(frm2, width=46, height=12, font="Times 10", yscrollcommand=self.scroll_Y.set)
        self.scroll_Y.config(command=self.txtSinopsis.yview)
            
        self.lblTitulo.grid(row=0, column=0, padx=5, pady=5, sticky='e')
        self.lblDirector.grid(row=1, column=0, padx=5, pady=5, sticky='e')
        self.lblDuracion.grid(row=2, column=0, padx=5, pady=5, sticky='e')
        self.lblSinopsis.grid(row=3, column=0, padx=5, pady=5, sticky='ne')
        
        self.entTitulo.grid(row=0, column=1, columnspan=2, padx=5, pady=5, sticky='e')
        self.entDirector.grid(row=1, column=1, columnspan=2, padx=5, pady=5, sticky='e')
        self.entDuracion.grid(row=2, column=1, columnspan=2, padx=5, pady=5, sticky='e')
        self.txtSinopsis.grid(row=3, column=1, padx=5, pady=5, sticky='e')
        self.scroll_Y.grid(row=3, column=2, sticky='ns')
        
        # ----------------------- Binds ----------------------------
        self.cboPeliculas.bind("<<ComboboxSelected>>", self.cargar_info)
        
        # --------- Eliminar esta linea. Ejemplo de como ajustar el texto de la sinopsis...
        # self.text_sinopsis = textwrap.wrap('Aca debe de insertar la sinopsis de la pelicula...', 52)
        # --------- Eliminar esta linea. Ejemplo de como insertar la informacion de la sinopsis
        # self.txtSinopsis.insert(tk.END, '\n'.join(self.text_sinopsis))
        
   # ---------------------------- METODOS DE LA CLASE -----------------------------------
    def query_movie_list(self):
        # Retorna una lista con los titulos de las peliculas
        
        return titulo
    
    
    def cargar_info(self, handle):
        # Retorna la información de una pelicula y muestra esta en los Entrys y Text de la App
        self.btnAbrirWeb.config(state='normal')  # Habilita el boton "Ir Web..."
        conn = sqlite3.connect("movies.db")
        cur = conn.cursor()
        comando = "SELECT titulo,director,duracion,sinopsis FROM tabla WHERE titulo = '"+ self.cboPeliculas.get()+ "'"
        cur.execute(comando)
        data = cur.fetchall()
        print(data[0][0],data[0][1],data[0][2],data[0][3])
        #print(self.cboPeliculas.get())       # Retorna el nombre del titulo
        
        self.entTitulo.config(state='normal')
        self.entDirector.config(state='normal')
        self.entDuracion.config(state='normal')
        self.txtSinopsis.config(state='normal')
        
        
        self.titulo.set(data[0][0])
        self.director.set(data[0][1])
        self.duracion.set(data[0][2])
        self.txtSinopsis.delete(1.0, tk.END)
        self.txtSinopsis.insert(1.0,data[0][3].strip())
        
        self.entTitulo.config(state='disable')
        self.entDirector.config(state='disable')
        self.entDuracion.config(state='disable')
        self.txtSinopsis.config(state='disable')
        
        
        
        
        
    
    
    def abrir_webbrowser(self):
        conn = sqlite3.connect("movies.db")
        cur = conn.cursor()
        comando = "SELECT links FROM tabla WHERE titulo = '"+ self.cboPeliculas.get()+ "'"
        cur.execute(comando)
        data = cur.fetchall()
        os.startfile("{}".format(data[0][0]))
        
        
        
root = tk.Tk()
app = App(root)
root.mainloop()

Jumper 2 
Doug Liman
 - 
        Secuela de 'Jumper' (2008), basada en la novela homónima de Steven Gould.
    
Jumper 2 
Doug Liman
 - 
        Secuela de 'Jumper' (2008), basada en la novela homónima de Steven Gould.
    
Top Gun: Maverick 
Joseph Kosinski
 - 
Después de más de 30 años de servicio como uno de los mejores aviadores de la Armada, Pete "Maverick" Mitchel se encuentra dónde siempre quiso estar, empujando los límites como un valiente piloto de prueba y esquivando el alcance en su rango que lo emplazaría en tierra.
 
Cuando se encuentra entrenando a un destacamento de graduados de Top Gun para una misión especializada, Maverick se encuentra con el Teniente Bradley Bradshaw, "Rooster", el hijo del difunto amigo de Maverick el Teniente Nick Bradshaw, "Goose".
 
Enfrentándose a un futuro incierto y a los fantasmas de su pasado, Maverick se ve envuelto en una confrontación con sus propios miedos más profundos, culminando en una misión que exige el máximo sacrificio de aquellos

---
## Parte 2: Pandas - 7

Se tiene la siguiente URL: https://en.wikipedia.org/wiki/World_population. En este sitio encontrarán un conjunto de tablas con información sobre la población mundial. La tabla de interes es la siguiente:

![](tabla_wikipedia.jpg)

Extraiga la información de la tabla hasta obtener el siguiente DataFrame:

![](pop_dataframe.jpg)

Una vez obtenido el DataFrame, obtenga el gráfico del crecimiento poblacional anual por region:

![](pop_plot.jpg)

---

Utilice todas las celdas que necesite. Insertelas con el icono del simbolo de "+"

In [7]:
import pandas as pd
import matplotlib.pyplot as plt
info =pd.read_html( "https://en.wikipedia.org/wiki/World_population")
tabla=info[8]




fil,col = tabla.shape
df = pd.DataFrame()



for i in range(fil):
    data = tabla.iloc[i]
    
    if data[0] == 'Region':
        df['Year'] = data[1:]
    else:
        df[data[0]] = data[1:]
df.set_index('Year',inplace=True) 
df.columns.name='Region'



df0 = df.drop('World',axis=1)


KeyError: "None of ['Year'] are in the columns"

In [4]:

tabla.set_index(tabla[0],inplace=True)
tabla1 = tabla.drop(0,axis=1)
tabla1


KeyError: 0

In [5]:
df

,2020,2019,2018,2017,2016,2015,2014,2013,2012,2011,...,2009,2008,2007,2006,2005,2004,2003,2002,2001,2000
Population,7795000000,7713468100,7631091040,7547858925,7464022049,7379797139,7295290765,7210581976,7125828059,7041194301,...,6872767093,6789088686,6705946610,6623517833,6541907027,6461159389,6381185114,6301773188,6222626606,6143493823
Yearly change,1.05%,1.08%,1.10%,1.12%,1.14%,1.16%,1.17%,1.19%,1.20%,1.21%,...,1.23%,1.24%,1.24%,1.25%,1.25%,1.25%,1.26%,1.27%,1.30%,1.31%
Net change,81330639,82377060,83232115,83836876,84224910,84506374,84708789,84753917,84633758,84370698,...,83678407,83142076,82428777,81610806,80747638,79974275,79411926,79146582,79132783,79254768
Density (P/Km²),52,52,51,51,50,50,49,48,48,47,...,47,46,45,44,44,43,43,42,42,41
Urban Population,4378900000,4299438618,4219817318,4140188594,4060652683,3981497663,3902831934,3824990329,3747842586,3671423872,...,3516830263,3439719128,3363609560,3289446226,3215905863,3143044892,3071743997,3001808223,2933078510,2868307513
Urban pop %,56%,56%,55%,55%,54%,54%,53%,53%,52%,52%,...,51%,50%,50%,50%,49%,48%,48%,47%,47%,46%


In [6]:
plt.style.use('ggplot')
df0.plot.line(title='Population',marker='o')
plt.grid(color='white')

NameError: name 'df0' is not defined

## RUBRICA DE EVALUACION
| Criterio de evaluación                                         | Nivel 0 | Nivel 1 | Nivel 2 
|----------------------------------------------------------------|---------|---------|---------
| Lee la informacion del URL y extrae la tabla a un DataFrame     | 0       | 1       | 2       
| Modifica el DataFrame hasta obtener la forma deseada           | 0       | 1       | 3       
| Obtiene el gráfico a partir del DataFrame                      | 0       | 1       | 2       

* Nivel 0: No existen elementos de calificación para este criterio
* Nivel 1: Cumple con los criterios de forma incompleta
* Nivel 2: Cumple con los criterios esperados en la evaluación